In [1]:
#División de train/test
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score
from pathlib import Path
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer   

In [2]:
#Preparamos X e y, revisando valores únicos y limpiando NaN en y

df = pd.read_csv('../data/raw/bank_marketing.csv') 
print('Valores únicos en Class:', df['Class'].unique())

# Normalizar valores a string y minúsculas para evitar problemas de formato
class_values = set(str(val).strip().lower() for val in df['Class'].unique())

# Mapear la variable objetivo según los valores únicos detectados
if class_values == {'1', '2'}:
    y = df['Class'].astype(int).map({1: 0, 2: 1})
elif class_values == {'no', 'yes'}:
    y = df['Class'].map({'no': 0, 'yes': 1})
elif class_values == {'0', '1'}:
    y = df['Class'].astype(int)
else:
    print('Advertencia: valores inesperados en Class, no se realiza mapeo binario.')
    y = df['Class']

# Eliminar columnas de fuga y target de X
X = df.drop(columns=['Class', 'V12']) if 'V12' in df.columns else df.drop(columns=['Class'])

# Limpiar NaN en y y ajustar X
if y.isnull().any():
    print(f"Advertencia: Se encontraron {y.isnull().sum()} valores NaN en y tras el mapeo. Se eliminarán esas filas.")
    X = X.loc[~y.isnull(), :].copy()
    y = y.dropna()

# Split de datos
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

print(X_train.shape, X_test.shape)
print("Proporción train:", y_train.mean().round(3))
print("Proporción test:", y_test.mean().round(3))

Valores únicos en Class: [1 2]
(36168, 15) (9043, 15)
Proporción train: 0.117
Proporción test: 0.117
(36168, 15) (9043, 15)
Proporción train: 0.117
Proporción test: 0.117


In [3]:
#Modelo baseline
#Codificar variables categóricas
X_train_enc = pd.get_dummies(X_train)
X_test_enc = pd.get_dummies(X_test)

# Alinear columnas de test con train
X_test_enc = X_test_enc.reindex(columns=X_train_enc.columns, fill_value=0)

# Escalar variables numéricas
scaler = StandardScaler()
num_cols = X_train_enc.select_dtypes(include=['float64', 'int64']).columns
X_train_enc[num_cols] = scaler.fit_transform(X_train_enc[num_cols])
X_test_enc[num_cols] = scaler.transform(X_test_enc[num_cols])

# Entrenar el modelo con los datos codificados y escalados
baseline = LogisticRegression(max_iter=1000)
baseline.fit(X_train_enc, y_train)

y_pred = baseline.predict(X_test_enc)
y_prob = baseline.predict_proba(X_test_enc)[:,1]

print("ROC-AUC:", roc_auc_score(y_test, y_prob).round(3))
print(classification_report(y_test, y_pred))

ROC-AUC: 0.772
              precision    recall  f1-score   support

           0       0.90      0.99      0.94      7985
           1       0.66      0.18      0.28      1058

    accuracy                           0.89      9043
   macro avg       0.78      0.58      0.61      9043
weighted avg       0.87      0.89      0.87      9043



In [4]:
#Guardar resultados iniciales
df = pd.read_csv('../data/raw/bank_marketing.csv') 

fig_dir = Path('../reports/figures')
fig_dir.mkdir(parents=True, exist_ok=True)

plt.figure(figsize=(8,4))
if 'Class' in df.columns:
    sns.countplot(x='Class', data=df)
    plt.title('Distribución de la variable objetivo')
    plt.tight_layout()
    plt.savefig(os.path.join(fig_dir, 'class_distribution.png'))
    plt.close()
else:
    print("La columna 'Class' no existe en el DataFrame.")

In [5]:
#Preprocesamiento y Feature Engineering
#cargar datos
df = pd.read_csv('../data/raw/bank_marketing.csv')

# Verificar valores únicos en la columna objetivo
if 'Class' in df.columns:
    print('Valores únicos en Class:', df['Class'].unique())
else:
    raise ValueError("La columna 'Class' no existe en el archivo CSV.")

# Mapear la variable objetivo a binaria si los valores son 'yes' y 'no'
if set(df['Class'].dropna().unique()) == {'yes', 'no'}:
    df['Class'] = df['Class'].map({'yes': 1, 'no': 0})
else:
    print("Advertencia: valores inesperados en Class, no se realiza mapeo binario.")

y = df['Class']

# Eliminar filas con NaN en la variable objetivo
if y.isnull().any():
    print(f"Advertencia: Se encontraron {y.isnull().sum()} valores NaN en y. Se eliminarán esas filas.")
    df = df.loc[~y.isnull(), :].copy()
    y = y.dropna()

# Eliminar columnas de fuga si existen
cols_to_drop = ['Class', 'duration']
X = df.drop(columns=[col for col in cols_to_drop if col in df.columns])

#Split estratificado
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

#Identificar variables categóricas y numéricas
num_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_features = X.select_dtypes(include=['object', 'category']).columns.tolist()

print('Variables numéricas:', num_features)
print('Variables categóricas:', cat_features)


Valores únicos en Class: [1 2]
Advertencia: valores inesperados en Class, no se realiza mapeo binario.
Variables numéricas: ['V1', 'V6', 'V10', 'V12', 'V13', 'V14', 'V15']
Variables categóricas: ['V2', 'V3', 'V4', 'V5', 'V7', 'V8', 'V9', 'V11', 'V16']
